In [1]:
# libraries

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics.pairwise import pairwise_distances
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


## Collaborative Filtering

Collaborative filtering is a method of making automatic predictions (filtering) about the interests of a user by collecting preferences from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on an issue, A is more likely to have B's opinion on a different issue than that of a randomly chosen person.

There are two types of collaborative filtering: user-based and item-based. User-based collaborative filtering is based on the similarity between users and item-based collaborative filtering is based on the similarity between items.

There are also two types of collaborative filtering algorithms: memory-based and model-based. Memory-based algorithms are based on statistical techniques and model-based algorithms are based on machine learning techniques. In this notebook, we will use model-based algorithms because they can scale with the number of users and items aligning with our goal of building a recommender system for a large user application.

## Item-based Collaborative Filtering

Let's start with an item-based collaborative filtering algorithm. Item-based collaborative filtering is often preferred over user-based collaborative filtering as it tend to perform better when there are many items and fewer users.

 We will calculate the similarity between items based on the ratings users have given to those items. We will use the cosine similarity to calculate the similarity between items. 



In [2]:
df_merged = pd.read_pickle('data/df_movies_cleaned.pkl')
df_ratings = pd.read_pickle('data/df_ratings_cleaned.pkl')

In [3]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24848104 entries, 0 to 24848103
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   userId            int64         
 1   movieId           int64         
 2   rating            Float64       
 3   timestamp         datetime64[ns]
 4   user_mean_rating  Float64       
 5   liked_by_user     boolean       
dtypes: Float64(2), boolean(1), datetime64[ns](1), int64(2)
memory usage: 1.0 GB


In [13]:
df_ratings.describe()

,userId,movieId,rating,timestamp,user_mean_rating
count,2.484810e+07,2.484810e+07,24848104.0,24848104,24848104.0
mean,1.350036e+05,1.621173e+04,3.528737,2007-02-21 23:13:44.095164416,3.528737
min,1.000000e+00,1.000000e+00,0.5,1995-01-09 11:46:44,0.5
25%,6.712600e+04,1.088000e+03,3.0,2001-06-09 05:22:35,3.2516
50%,1.351340e+05,2.670000e+03,3.5,2006-06-16 19:53:25.500000,3.55
75%,2.026420e+05,6.711000e+03,4.0,2013-02-19 17:38:24.249999872,3.836879
max,2.708960e+05,1.762750e+05,5.0,2017-08-04 06:57:50,5.0
std,7.817512e+04,3.135802e+04,1.060048,NaN,0.46693


To safe computational time, we will use a subset of the data.

In [4]:
df_ratings['timestamp'] = pd.to_datetime(df_ratings['timestamp'])
df_ratings = df_ratings.sort_values('timestamp')
df_ratings_subset = df_ratings[df_ratings['timestamp'] > '2017-01-01']
df_ratings_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1270216 entries, 23070950 to 16502857
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   userId            1270216 non-null  int64         
 1   movieId           1270216 non-null  int64         
 2   rating            1270216 non-null  Float64       
 3   timestamp         1270216 non-null  datetime64[ns]
 4   user_mean_rating  1270216 non-null  Float64       
 5   liked_by_user     1270216 non-null  boolean       
dtypes: Float64(2), boolean(1), datetime64[ns](1), int64(2)
memory usage: 63.0 MB


In [17]:
# Train/Test Split
train_data, test_data = train_test_split(df_ratings_subset, test_size=0.2, random_state=42)

# User-Item Matrix for Training
user_item_matrix_train = train_data.pivot_table(index='userId', columns='movieId', values='rating')

# Item-Item Similarity Matrix
item_similarity = cosine_similarity(user_item_matrix_train.fillna(0).T)
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix_train.columns, columns=user_item_matrix_train.columns)

print(item_similarity_df)
print(item_similarity_df.info())


movieId    1         2         3         4         5         6         7       \
movieId                                                                         
1        1.000000  0.291622  0.104701  0.041914  0.137984  0.192833  0.083149   
2        0.291622  1.000000  0.137777  0.032426  0.167501  0.170788  0.118674   
3        0.104701  0.137777  1.000000  0.094457  0.272469  0.137315  0.150091   
4        0.041914  0.032426  0.094457  1.000000  0.072964  0.042718  0.024702   
5        0.137984  0.167501  0.272469  0.072964  1.000000  0.072802  0.205986   
...           ...       ...       ...       ...       ...       ...       ...   
176267   0.025938  0.033563  0.000000  0.000000  0.000000  0.000000  0.110065   
176269   0.000000  0.039157  0.000000  0.000000  0.000000  0.000000  0.000000   
176271   0.019454  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
176273   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
176275   0.000000  0.000000 

Let's use Singular Value Decomposition (SVD) on our user-item ratings matrix. SVD helps in extracting latent factors that explain observed ratings, efficiently reducing data dimensionality while preserving essential information. This significantly speeds up calculations, making the process of predicting ratings more efficient, especially when dealing with large dataset like ours. 

In [16]:
user_item_matrix_train = user_item_matrix_train.fillna(0).astype(float)
mean_user_rating = user_item_matrix_train.mean(axis=1)
R_demeaned = user_item_matrix_train.sub(mean_user_rating, axis='index')

R_demeaned_matrix = R_demeaned.values

# SVD 
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned_matrix, k=50)  

sigma_matrix = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma_matrix), Vt) + mean_user_rating.values.reshape(-1, 1)

preds_df = pd.DataFrame(all_user_predicted_ratings, index=user_item_matrix_train.index, columns=user_item_matrix_train.columns)

actual = []
predicted = []

for index, row in test_data.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']
    if user_id in preds_df.index and movie_id in preds_df.columns:
        actual_rating = row['rating']
        predicted_rating = preds_df.loc[user_id, movie_id]

        actual.append(actual_rating)
        predicted.append(predicted_rating)

rmse = np.sqrt(mean_squared_error(actual, predicted))
print(f'RMSE: {rmse}')


RMSE: 2.904032531436712


Given a 1-to-5 scale, an RMSE of 2.904 is relatively high, indicating that the predictions can be quite far off from the actual ratings. 